# Laboratorio 1 (AlpesHearth): Exploración, preparación y regresión lineal

Este notebook está diseñado para cumplir el enunciado completo:

1. Exploración y perfilamiento (calidad: completitud, unicidad, consistencia, validez).
2. Limpieza y preparación con justificación.
3. Dos modelos de regresión lineal con **pipelines** y preparación distinta.
4. Tabla comparativa en test con **RMSE, MAE y R2**.
5. Importancia de variables con base en el mejor modelo (coeficientes).
6. Validación de supuestos (gráficos de residuos) para apoyar interpretación.
7. Predicción sobre el archivo de test no etiquetado y exportación en CSV.

Nota importante (causa típica de R2 negativo):
- La columna **`Blood Pressure (mmHg)`** viene como texto tipo `"120/80"`. Si se deja como categórica, crea miles de dummies y generaliza muy mal.
- Aquí la usamos solo para **completar** `Systolic BP` y `Diastolic BP`, y luego la excluimos del modelado.


## 0. Imports y constantes

In [ ]:

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.base import BaseEstimator, TransformerMixin

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

RANDOM_STATE = 42
TEST_SIZE = 0.25

TARGET = "CVD Risk Score"
AUX_LABEL = "CVD Risk Level"


## 1. Cargar datos y diccionario

Punto clave: `Datos Test Lab 1.csv` usa `;` y el train usa `,`. Esta celda detecta el separador automáticamente.


In [ ]:

def read_csv_auto_sep(path: str) -> pd.DataFrame:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        first_line = f.readline()
    sep = ";" if first_line.count(";") > first_line.count(",") else ","
    return pd.read_csv(path, sep=sep)

train_path = "Datos Lab 1.csv"
test_path  = "Datos Test Lab 1.csv"
dicc_path  = "DiccPacientes.xlsx"

train_raw = read_csv_auto_sep(train_path)
test_raw  = read_csv_auto_sep(test_path)
dicc_raw  = pd.read_excel(dicc_path)
dicc_raw.columns = [c.strip() for c in dicc_raw.columns]

print("Train shape:", train_raw.shape)
print("Test  shape:", test_raw.shape)
display(dicc_raw.head(10))

print("Columnas extra en train (vs test):", sorted(list(set(train_raw.columns) - set(test_raw.columns))))


## 2. Exploración inicial

In [ ]:

display(train_raw.head())
display(test_raw.head())

print("Tipos de datos (train):")
display(train_raw.dtypes)

print("Describe numérico (train):")
display(train_raw.select_dtypes(include=[np.number]).describe())

print("Describe del target (train):")
display(train_raw[TARGET].describe())


## 3. Perfilamiento de calidad (completitud, unicidad, consistencia, validez)

In [ ]:

# Completitud
missing_train = (train_raw.isna().mean() * 100).sort_values(ascending=False)
missing_test  = (test_raw.isna().mean() * 100).sort_values(ascending=False)

print("Missing % (train) - top 15")
display(missing_train.head(15))

print("Missing % (test) - top 15")
display(missing_test.head(15))

print("Filas en train con target missing:", train_raw[TARGET].isna().sum())

# Unicidad
print("Duplicados exactos train:", train_raw.duplicated().sum())
print("Duplicados exactos test :", test_raw.duplicated().sum())


### 3.1 Consistencia (categóricas)

In [ ]:

CAT_COLS = [
    "Sex",
    "Smoking Status",
    "Diabetes Status",
    "Physical Activity Level",
    "Family History of CVD",
    "Blood Pressure Category",
]

for c in CAT_COLS:
    print("\n---", c, "---")
    display(train_raw[c].astype(str).value_counts().head(20))


### 3.2 Validez (valores imposibles)

In [ ]:

print("LDL negativo (train):", (train_raw["Estimated LDL (mg/dL)"] < 0).sum())
print("Colesterol total negativo (train):", (train_raw["Total Cholesterol (mg/dL)"] < 0).sum())

print("Peso <= 0 (train):", (train_raw["Weight (kg)"] <= 0).sum(skipna=True))
print("Altura (m) <= 0 (train):", (train_raw["Height (m)"] <= 0).sum(skipna=True))
print("Edad <= 0 (train):", (train_raw["Age"] <= 0).sum(skipna=True))


## 4. Limpieza y preparación (justificada)

Reglas:
- Quitar duplicados exactos.
- Quitar filas sin `CVD Risk Score` en train.
- LDL negativo y colesterol total negativo se vuelven NaN.
- Completar `Systolic BP` y `Diastolic BP` desde `Blood Pressure (mmHg)` si es posible.
- Normalizar categóricas con `strip()`.

Importante:
- `Blood Pressure (mmHg)` se usa para completar BP numérica, pero **NO** se usa como predictor.


In [ ]:

def parse_bp_series(series: pd.Series):
    sys_vals, dia_vals = [], []
    for v in series.astype(str):
        m = re.match(r"^\s*(\d+)\s*/\s*(\d+)\s*$", v)
        if m:
            sys_vals.append(float(m.group(1)))
            dia_vals.append(float(m.group(2)))
        else:
            sys_vals.append(np.nan)
            dia_vals.append(np.nan)
    return pd.Series(sys_vals, index=series.index), pd.Series(dia_vals, index=series.index)

def normalize_categories(df: pd.DataFrame, cat_cols: list[str]) -> pd.DataFrame:
    df = df.copy()
    for c in cat_cols:
        df[c] = df[c].astype("string").str.strip()
    return df

def clean_common(df: pd.DataFrame, cat_cols: list[str] = None) -> pd.DataFrame:
    if cat_cols is None:
        cat_cols = CAT_COLS

    df = df.copy()
    df = normalize_categories(df, cat_cols)

    # Validez
    df.loc[df["Estimated LDL (mg/dL)"] < 0, "Estimated LDL (mg/dL)"] = np.nan
    df.loc[df["Total Cholesterol (mg/dL)"] < 0, "Total Cholesterol (mg/dL)"] = np.nan

    # Completar BP numérica desde el string
    if "Blood Pressure (mmHg)" in df.columns:
        sys_bp, dia_bp = parse_bp_series(df["Blood Pressure (mmHg)"])
        df["Systolic BP"] = df["Systolic BP"].fillna(sys_bp)
        df["Diastolic BP"] = df["Diastolic BP"].fillna(dia_bp)

    return df

train = train_raw.drop_duplicates().dropna(subset=[TARGET]).copy()
train = clean_common(train)

test = test_raw.drop_duplicates().copy()
test = clean_common(test)

print("Train limpio shape:", train.shape)
print("Test  limpio shape:", test.shape)

print("LDL negativo (train limpio):", (train["Estimated LDL (mg/dL)"] < 0).sum(skipna=True))
print("Colesterol total negativo (train limpio):", (train["Total Cholesterol (mg/dL)"] < 0).sum(skipna=True))


## 5. Visualizaciones útiles (target y correlaciones)

In [ ]:

plt.figure(figsize=(6,4))
sns.histplot(train[TARGET], kde=True)
plt.title("Distribución de CVD Risk Score")
plt.xlabel(TARGET)
plt.ylabel("Frecuencia")
plt.show()

corr = train.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(10,6))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Heatmap de correlación (numéricas)")
plt.show()

target_corr = corr[TARGET].drop(TARGET).abs().sort_values(ascending=False)
display(target_corr.head(10))


## 6. Definir X, y y partición train-test (enunciado: semilla 42 y 25%)

Decisión crítica (para evitar R2 negativo):
- Excluimos `Blood Pressure (mmHg)` del modelado porque es texto con alta cardinalidad.


In [ ]:

DROP_ALWAYS = ["Patient ID", "Date of Service", AUX_LABEL, "Blood Pressure (mmHg)"]

X = train.drop(columns=[TARGET] + DROP_ALWAYS).copy()
y = train[TARGET].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print("X_train:", X_train.shape, "X_test:", X_test.shape)


## 7. Dos modelos (pipelines) con preparación distinta

Modelo 1 (baseline OLS):
- Imputación + OneHot + LinearRegression

Modelo 2 (RidgeCV):
- Imputación + escalado numérico + OneHot + RidgeCV (regresión lineal regularizada)

Por qué Ridge aquí:
- Sigue siendo regresión lineal.
- Ayuda con multicolinealidad y alta dimensionalidad por dummies, mejora generalización.


In [ ]:

def build_preprocess(X_sample: pd.DataFrame, scale_numeric: bool) -> ColumnTransformer:
    num_cols = X_sample.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X_sample.columns if c not in num_cols]

    num_steps = [("imputer", SimpleImputer(strategy="median"))]
    if scale_numeric:
        num_steps.append(("scaler", StandardScaler()))

    preprocess = ColumnTransformer(
        transformers=[
            ("num", Pipeline(num_steps), num_cols),
            ("cat", Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore", drop="first")),
            ]), cat_cols),
        ],
        remainder="drop"
    )
    return preprocess

# Modelo 1: OLS
model_1 = Pipeline([
    ("preprocess", build_preprocess(X_train, scale_numeric=False)),
    ("reg", LinearRegression()),
])

# Modelo 2: RidgeCV (elige alpha automáticamente)
alphas = np.logspace(-3, 3, 25)
model_2 = Pipeline([
    ("preprocess", build_preprocess(X_train, scale_numeric=True)),
    ("reg", RidgeCV(alphas=alphas)),
])


## 8. Evaluación cuantitativa (RMSE, MAE, R2) y baseline del promedio

In [ ]:

def eval_model(model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr, y_tr)
    pred = model.predict(X_te)
    rmse = np.sqrt(mean_squared_error(y_te, pred))
    mae  = mean_absolute_error(y_te, pred)
    r2   = r2_score(y_te, pred)
    return rmse, mae, r2

# Baseline: predecir el promedio del target
baseline_pred = np.repeat(y_train.mean(), len(y_test))
baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))
baseline_mae  = mean_absolute_error(y_test, baseline_pred)
baseline_r2   = r2_score(y_test, baseline_pred)

rmse1, mae1, r21 = eval_model(model_1, X_train, y_train, X_test, y_test)
rmse2, mae2, r22 = eval_model(model_2, X_train, y_train, X_test, y_test)

results = pd.DataFrame([
    {"Modelo": "Baseline (promedio)", "RMSE": baseline_rmse, "MAE": baseline_mae, "R2": baseline_r2},
    {"Modelo": "Modelo 1 (OLS)", "RMSE": rmse1, "MAE": mae1, "R2": r21},
    {"Modelo": "Modelo 2 (RidgeCV)", "RMSE": rmse2, "MAE": mae2, "R2": r22},
]).sort_values("RMSE", ascending=True)

display(results)

best_row = results[results["Modelo"].str.contains("Modelo")].sort_values("RMSE").iloc[0]
best_name = best_row["Modelo"]
best_model = model_1 if best_name == "Modelo 1 (OLS)" else model_2

print("Mejor modelo (por RMSE):", best_name)

if best_name == "Modelo 2 (RidgeCV)":
    print("Alpha seleccionado (RidgeCV):", best_model.named_steps["reg"].alpha_)


## 9. Importancia de variables: coeficientes del mejor modelo

Extraemos:
- nombres de features (después del preprocesamiento)
- coeficientes del regresor (OLS o Ridge)

Se ordena por valor absoluto para reportar variables más influyentes.


In [ ]:

best_model.fit(X_train, y_train)

preprocess_step = best_model.named_steps["preprocess"]
feature_names = preprocess_step.get_feature_names_out()

coefs = best_model.named_steps["reg"].coef_

coef_df = (
    pd.DataFrame({"Feature": feature_names, "Coef": coefs})
      .assign(AbsCoef=lambda d: d["Coef"].abs())
      .sort_values("AbsCoef", ascending=False)
)

display(coef_df.head(25))


## 10. Validación visual de supuestos (residuos)

Para apoyar interpretación:
- residuos vs predicción
- histograma de residuos
- QQ plot

Esto no prueba formalmente, pero es suficiente para la etapa cualitativa del laboratorio.


In [ ]:

y_pred = best_model.predict(X_test)
residuals = y_test - y_pred

plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.4)
plt.axhline(0)
plt.title("Residuos vs Predicción")
plt.xlabel("Predicción")
plt.ylabel("Residuo (y - y_hat)")
plt.show()

plt.figure(figsize=(6,4))
sns.histplot(residuals, kde=True)
plt.title("Histograma de residuos")
plt.xlabel("Residuo")
plt.ylabel("Frecuencia")
plt.show()

import scipy.stats as stats
plt.figure(figsize=(6,4))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("QQ-plot de residuos")
plt.show()


## 11. Entrenar el mejor modelo con TODO el train y predecir el test no etiquetado

Reglas importantes:
- Limpiar el test original (para construir X_submit).
- Exportar usando el **test_raw** (para conservar filas y orden exacto del archivo entregable).
- Alinear columnas con `reindex` para evitar errores por columnas faltantes.


In [ ]:

# 1) Train completo (ya limpio) para entrenar final
X_full = train.drop(columns=[TARGET] + DROP_ALWAYS).copy()
y_full = train[TARGET].copy()

# 2) Partimos del test ORIGINAL para conservar filas y orden del archivo entregable
test_submit = clean_common(test_raw.copy(), CAT_COLS)

# 3) Armamos X_submit y lo alineamos con X_full
X_submit = test_submit.drop(columns=DROP_ALWAYS, errors="ignore").copy()
X_submit = X_submit.reindex(columns=X_full.columns)

# 4) Entrenar y predecir
best_model.fit(X_full, y_full)
test_pred = best_model.predict(X_submit)

# 5) Exportar el archivo final con el mismo formato del test (separador ;)
out = test_raw.copy()
out[TARGET] = test_pred

output_path = "Datos Test Lab 1.csv"
out.to_csv(output_path, index=False, sep=";")

print("Archivo generado para entrega:", output_path)
display(out.head())


## 12. Preguntas de análisis de resultados (responde aquí)

Responde directamente las preguntas del enunciado usando tus salidas:

1) Coeficientes del mejor modelo (usa `coef_df`).
2) Mejor rendimiento (usa `results`) e interpretación de RMSE, MAE, R2.
3) Variables seleccionadas (top de `coef_df`) e interpretación en el contexto AlpesHearth.
4) Representación matemática:
   - OLS: β = (XᵀX)⁻¹Xᵀy
   - Ridge: β = (XᵀX + αI)⁻¹Xᵀy
5) Dos sesgos: selección (muestra no representativa), medición (laboratorios con error), etc.
